# Charity Scraper
### The Plan:
1. Stage 1 scraping: Scrape Each Charity's register_index URL from the main search page
1. Stage 2 scraping: Open each charity and scrape characteristics
1. Stage 3 analysis:
  - clean date data
  - Use generated charity_index to filter for websites with desired characteristics
    * income
    * reporting
    * cause helped
    * has website and charity_url

## Stage 1 Scraping
#### todo:
- scrape 10 at a time
- rerun
- add placeholder headings

In [2]:
# Initialisation and loadup
from http import server
import pandas as pd
import os
import time
import numpy as np
import copy

import selenium as sl
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
driverpath=ChromeDriverManager().install()
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
size = 'Small'

In [26]:
# initialise the pandas dataframe
columns = ["Index_Link", "Legal_Name", "Town/Suburb","State", "Status", "Size","ABN","Website","Revenue","Expenses","Cause/s","Last_report_date","ref_religion"]
try:
    os.mkdir('s1scrapedata')
except:
    None

try:
    os.mkdir('s2scrapedata')
except:
    None

In [1]:

def getpageurl(p):
    lefturl="https://www.acnc.gov.au/charity/charities?items_per_page=100&page="
    pagenumber=str(p)
    righturl="&f[]=size%3A"+size
    
    url = lefturl+pagenumber+righturl #pre filtered url for small businesses and to show 100 results - Page 1
    return url

In [28]:
def extract_page_data(d):
    try:
        WebDriverWait(d,90).until(EC.presence_of_element_located((By.TAG_NAME,"tbody")))
        time.sleep(1)
        table=d.find_element(by=By.TAG_NAME, value="tbody")
        rows=table.find_elements(by=By.TAG_NAME,value="tr")
        
        for row in rows:
            link   = row.find_element(by=By.TAG_NAME,value="a").get_attribute("href")
            name   = row.find_elements(by=By.TAG_NAME, value="td")[0].text
            town   = row.find_elements(by=By.TAG_NAME, value="td")[1].text
            state  = row.find_elements(by=By.TAG_NAME, value="td")[2].text
            status = row.find_elements(by=By.TAG_NAME, value="td")[3].text
            size   = row.find_elements(by=By.TAG_NAME, value="td")[4].text
            abn    = row.find_elements(by=By.TAG_NAME, value="td")[5].text

            row_data = pd.DataFrame([link,name,town,state,status,size,abn,None,None,None,None,None,None], index=columns).transpose()
            try:
                page_data = pd.concat([page_data,row_data],ignore_index=True)
            except:
                page_data = row_data
        
        return page_data
    except:
        print('Table could not be found on page'+str(page))
        return None 


    

In [29]:
#   df=pd.DataFrame([0,1,'b'], index=[10,20,30]).transpose()
#   b=pd.DataFrame([2,3,5], index=[10,20,30]).transpose()
#   df=pd.concat([df,b],ignore_index=True)
#   print(df)
#   df=pd.concat([df,copy.deepcopy(b)],ignore_index=True)
#   print(df)
#   
#   df.to_csv('./folder/file.csv')

In [30]:
a=np.array([0,0,0])
print(a)
a=np.vstack((a,[1,2,3]))
print(a)


[0 0 0]
[[0 0 0]
 [1 2 3]]


In [31]:
def open_drivers(num_drivers=10):
    drivers =[]
    for i in range(num_drivers):
        drivers.append(webdriver.Chrome(service=Service(driverpath)))
    return drivers

In [32]:
def run_scraper():
    drivers = open_drivers(5)
    try:
        #navigate into pages and scrape
        page=0
        totalpages= 353 - 1 #actual pages-1
        try:
            ScrapedSearchPages = np.load('./s1scrapedata/scrapedpageindex.npy')
        except:
            ScrapedSearchPages = np.array([])

        tempdf = pd.DataFrame()

        while page < totalpages:
            #load up sites
            pagecopy = copy.deepcopy(page)
            for d in drivers:
                while pagecopy in ScrapedSearchPages:
                    pagecopy+=1
                if pagecopy < totalpages:
                    d.get(getpageurl(pagecopy))        
                    pagecopy+=1

            ## scraping
            for D in drivers:
                while page in ScrapedSearchPages:
                    page +=1
                page_data = extract_page_data(d=D)
                tempdf=pd.concat((tempdf,page_data),ignore_index=True)
                ScrapedSearchPages=np.append(ScrapedSearchPages, page)

                if (page+1) % 15 == 0:
                    np.save('./s1scrapedata/scrapedpageindex.npy',ScrapedSearchPages)
                    tempdf.to_csv('./s1scrapedata/S1scrape_topage'+str(page+1)+'.csv')
                    tempdf = pd.DataFrame()
    
                page += 1
                print("Progress: "+str(page)+"/"+str(totalpages+1))

        #final Save
        np.save('./s1scrapedata/scrapedpageindex.npy',ScrapedSearchPages)
        tempdf.to_csv('./s1scrapedata/S1scrape_topage'+str(page+1)+'.csv')
        tempdf = pd.DataFrame()


        #kill drivers
        for d in drivers:
            d.quit()

    except Exception as e:
        for d in drivers:
            d.quit()
        raise e
    

In [34]:
run_scraper()

Progress: 196/353
Progress: 197/353
Progress: 198/353
Progress: 199/353
Progress: 200/353
Progress: 201/353
Progress: 202/353
Progress: 203/353
Progress: 204/353
Progress: 205/353
Progress: 206/353
Progress: 207/353
Progress: 208/353
Progress: 209/353
Progress: 210/353
Progress: 211/353
Progress: 212/353
Progress: 213/353
Progress: 214/353
Progress: 215/353
Progress: 216/353
Progress: 217/353
Progress: 218/353
Progress: 219/353
Progress: 220/353
Progress: 221/353
Progress: 222/353
Progress: 223/353
Progress: 224/353
Progress: 225/353
Progress: 226/353
Progress: 227/353
Progress: 228/353
Progress: 229/353
Progress: 230/353
Progress: 231/353
Progress: 232/353
Progress: 233/353
Progress: 234/353
Progress: 235/353
Progress: 236/353
Progress: 237/353
Progress: 238/353
Progress: 239/353
Progress: 240/353
Progress: 241/353
Progress: 242/353
Progress: 243/353
Progress: 244/353
Progress: 245/353
Progress: 246/353
Progress: 247/353
Progress: 248/353
Progress: 249/353
Progress: 250/353
Progress: 